In [ ]:
from preprocessing.Preprocessing import Preprocessing
import pandas as pd
import traceback
import sys

data_path = '/Users/User/Desktop/radiology/data/'
preprocessing = Preprocessing(data_folder = data_path)
data =  pd.read_csv(data_path + 'Radiology.csv')
data['cleaned_report'] = data['توضیحات پزشک'].apply(lambda x: preprocessing.cleaner(x))
data['cleaned_report'] = data['cleaned_report'].apply(lambda x: preprocessing.complete_normalization(x))
data['tokenized_report'] = data['cleaned_report'].apply(lambda x: preprocessing.sentence_tokenizer(x))
data.to_excel(data_path + 'cleaned_tokenized_report.xlsx')

In [9]:
persian_diseases = list(pd.read_csv(data_path + 'persian_terms.csv')['Term'])
english_diseases = ['fx','Avulsion fracture', 'Chronic Lymphocytic Thyroiditis', 'clt', 'djd', 'fracture', 'intertrochanteric fracture', 'osgood Schlatter', 'calcification', 'adenoma']
diseases = persian_diseases + english_diseases
body_parts = list(pd.read_csv(data_path + 'body_parts.csv')['Term'])

In [ ]:
diseases = list(set([preprocessing.complete_normalization(x) for x in diseases]))
body_parts = list(set([preprocessing.complete_normalization(x) for x in body_parts]))

In [20]:
sentences = pd.read_excel(data_path + 'unique_sentences.xlsx')

def find_body_parts_len(sent):
    sent = set(sent.split())
    return len(sent.intersection(set(body_parts)))

def find_body_parts(sent):
    sent = set(sent.split())
    return sent.intersection(set(body_parts))

sentences['num_body_parts'] = sentences['sentence'].apply(lambda x:find_body_parts_len(x))
sentences['body_parts'] = sentences['sentence'].apply(lambda x:find_body_parts(x))

In [26]:
sentences[sentences['num_body_parts'] > 2]

,توضیحات پزشک,cleaned_report,tokenized_report
0,گزارش رادیولوژی  سند : FO-MR-78...,قفسه صدری دیگر 1 فیلم رادیوگرافی ریه: اندازه ق...,"[قفسه صدری دیگر 1 فیلم رادیوگرافی ریه tag ., ..."
1,گزارش رادیولوژی  سند : FO-MR-78...,ستون فقرات گردن 2 جهت رخ نیمرخ رادیوگرافی دنده...,[ستون فقرات گردن 2 جهت رخ نیمرخ رادیوگرافی دند...
2,گزارش رادیولوژی  سند : FO-MR-78...,استخوانهای مخصوص بینی نمای 1 فیلم رادیوگرافی ب...,[استخوانهای مخصوص بینی نمای 1 فیلم رادیوگرافی ...
3,گزارش رادیولوژی  سند : FO-MR-78...,پا 2 جهت رادیوگرافی: مفاصل استخوانهای project ...,"[پا 2 جهت رادیوگرافی tag ., مفاصل استخوانهای ..."
4,گزارش رادیولوژی  سند : FO-MR-78...,استخوانهای دست 1 جهت رادیوگرافی: مفاصل استخوان...,"[استخوانهای دست 1 جهت رادیوگرافی tag ., مفاصل..."
...,...,...,...
995,گزارش رادیولوژی  سند : FO-MR-78...,شانه 1 جهت استخوان اسکاپولا، کلاویکل، اکرومیوک...,[شانه 1 جهت استخوان اسکاپولا، کلاویکل، اکرومیو...
996,گزارش رادیولوژی  سند : FO-MR-78...,قفسه صدری دیگر 1 فیلم رادیوگرافی لگن خاصره هرف...,[قفسه صدری دیگر 1 فیلم رادیوگرافی لگن خاصره هر...
997,گزارش رادیولوژی  سند : FO-MR-78...,قفسه صدری دیگر 1 فیلم رادیوگرافی پا 2 اکسپوز 1...,[قفسه صدری دیگر 1 فیلم رادیوگرافی پا 2 اکسپوز ...
998,گزارش رادیولوژی  سند : FO-MR-78...,لگن خاصره هرفیلم رادیوگرافی ران 2 اکسپوز 1 فیل...,[لگن خاصره هرفیلم رادیوگرافی ران 2 اکسپوز 1 فی...


In [42]:
import numpy as np

def bodypart_disease_alignment(sent):
    print(sent)
    result = []
    sent = sent.split()
    
    shared_bodyparts = list(set(sent).intersection(body_parts))
    index_bps = [sent.index(x) for x in shared_bodyparts]
    shared_diseases  = list(set(sent).intersection(diseases))
    print('shared_diseases', shared_diseases)
    index_diseases = [sent.index(x) for x in shared_diseases]
    print('index_diseases', index_diseases)
    for index, d_index in enumerate(index_diseases):
        
        tmp_arr = np.abs(np.array(index_bps) - d_index)
        bp_min_index = np.argmin(tmp_arr)
        result.append({'disease':shared_diseases[index],'body_part':shared_bodyparts[bp_min_index]})
        
    return result

    

In [43]:
bodypart_disease_alignment(sentences[sentences['num_body_parts'] > 2].iloc[1]['sentence'])

 FX میانی دیافیز تیبیا و فیبولای راست همراه با جابجایی Significant قطعات در FX فیبولا رویت شد 
shared_diseases ['جابجایی']
index_diseases [9]


[{'disease': 'جابجایی', 'body_part': 'فیبولای'}]